The first task is setting some context for later weeks by understanding how popular the Flow Card is. Our stakeholder would like two data sets about our passengers. One data set for card users and one data set for those who don't use the card. 
Input
There is one input file. You can download it from here.


 - Requirements
 - Input the data
 - Split the Flight Details field to form:
 - Date 
 - Flight Number
 - From
 - To
 - Class
 - Price
 - Convert the following data fields to the correct data types:
 - Date to a date format
 - Price to a decimal value
 - Change the Flow Card field to Yes / No values instead of 1 / 0
 - Create two tables, one for Flow Card holders and one for non-Flow Card holders
 - Output the data sets
 - Output
 - Both outputs have the same data structure:

9 data fields:
 - Date
 - Flight Number
 - From
 - To
 - Class
 - Price
 - Flow Card?
 - Bags Checked
 - Meal Type

In [60]:
import os
import pandas as pd
import numpy as np

In [61]:
# Inpor the data
df = pd.read_csv("PD 2024 Wk 1 Input.csv")
df.columns = [i.lower().strip().replace(' ','_').replace('?','') for i in df.columns]

df.head()

,flight_details,flow_card,bags_checked,meal_type
0,2024-07-22//PA010//Tokyo-New York//Economy//2380,1,0,Egg Free
1,2024-09-28//PA008//Perth-New York//Economy//1855,0,2,Vegetarian
2,2024-04-20//PA002//New York-London//Economy//3490,1,1,Vegan
3,2024-01-23//PA010//Tokyo-New York//Premium Eco...,1,1,Vegetarian
4,2024-10-01//PA008//Perth-New York//Business Cl...,0,0,Vegetarian


In [62]:
exp = df.flight_details.str.split('//',expand=True)
exp.columns = ['date','flight_number','2','class','price']

exp.head()

,date,flight_number,2,class,price
0,2024-07-22,PA010,Tokyo-New York,Economy,2380
1,2024-09-28,PA008,Perth-New York,Economy,1855
2,2024-04-20,PA002,New York-London,Economy,3490
3,2024-01-23,PA010,Tokyo-New York,Premium Economy,825
4,2024-10-01,PA008,Perth-New York,Business Class,634.79999999999995


In [63]:
exp['from'] = exp['2'].str.split('-').str[0]
exp['to'] = exp['2'].str.split('-').str[1]
exp.drop(columns=['2'],inplace=True)

exp.head()

,date,flight_number,class,price,from,to
0,2024-07-22,PA010,Economy,2380,Tokyo,New York
1,2024-09-28,PA008,Economy,1855,Perth,New York
2,2024-04-20,PA002,Economy,3490,New York,London
3,2024-01-23,PA010,Premium Economy,825,Tokyo,New York
4,2024-10-01,PA008,Business Class,634.79999999999995,Perth,New York


In [64]:
df=df.merge(exp,left_index=True,right_index=True).drop(columns=['flight_details'])[['date',
                                                 'flight_number',
                                                 'from',
                                                 'to',
                                                 'class',
                                                 'price',
                                                 'flow_card',
                                                 'bags_checked',
                                                 'meal_type'
                                                 ]].copy()

df.head()


,date,flight_number,from,to,class,price,flow_card,bags_checked,meal_type
0,2024-07-22,PA010,Tokyo,New York,Economy,2380,1,0,Egg Free
1,2024-09-28,PA008,Perth,New York,Economy,1855,0,2,Vegetarian
2,2024-04-20,PA002,New York,London,Economy,3490,1,1,Vegan
3,2024-01-23,PA010,Tokyo,New York,Premium Economy,825,1,1,Vegetarian
4,2024-10-01,PA008,Perth,New York,Business Class,634.79999999999995,0,0,Vegetarian


In [65]:
df['date']=pd.to_datetime(df['date'])
df['price']=df['price'].astype('float64')

flow_map={
  1:'Yes',
  0:'No'
}


df['flow_card']=df['flow_card'].map(flow_map)

df.head()

,date,flight_number,from,to,class,price,flow_card,bags_checked,meal_type
0,2024-07-22,PA010,Tokyo,New York,Economy,2380.0,Yes,0,Egg Free
1,2024-09-28,PA008,Perth,New York,Economy,1855.0,No,2,Vegetarian
2,2024-04-20,PA002,New York,London,Economy,3490.0,Yes,1,Vegan
3,2024-01-23,PA010,Tokyo,New York,Premium Economy,825.0,Yes,1,Vegetarian
4,2024-10-01,PA008,Perth,New York,Business Class,634.8,No,0,Vegetarian


In [68]:
df[df.flow_card=='Yes'].to_csv('With Cards.csv',index=False)
df[df.flow_card=='No'].to_csv('Without Cards.csv',index=False)